In [2]:
import pandas as pd
labels_df = pd.read_excel('MIT_AST_label_map.xlsx')

In [21]:
labels_df.head()

,Unnamed: 0,label,source
0,0,Speech,human
1,1,"Male speech, man speaking",human
2,2,"Female speech, woman speaking",human
3,3,"Child speech, kid speaking",human
4,4,Conversation,human


In [28]:
dd = labels_df['label'].iloc[1]
type(dd)

str

In [ ]:
# extract row with label 'Whistling'

ss = labels_df['label']
print(ss)

In [50]:
from test_mit_ast import MIT_AST_model
cfile = 'er_file_2023_09_04_7_13_08.wav'
bfile = "few_garden_files/er_file_2023_09_02_16_45_18.wav"
afile ="speech_whistling2.wav"
model = MIT_AST_model()
res = model.classify(bfile)
print(res)

Caw


In [113]:
#analyze files form folder and save results in a tuple: file name, string with labels
import os
def analyze_files_ast(folder):    
    files = os.listdir(folder)
    results = []
    for file in files:
        # Construct the full path to the file
        file_path = os.path.join(folder, file)
        try:
            result = model.classify(file_path)
            if len(result) > 0:
                results.append((file_path, result))
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return results

In [115]:
file_path = './gardenBigFolder/garden_01012024/0'
test_file_path = 'big_test_folder/few_garden_files/'
res = analyze_files_ast(test_file_path)
print(res)

[('big_test_folder/few_garden_files/er_file_2023_09_04_7_49_38.wav', 'Chirp, tweet'), ('big_test_folder/few_garden_files/er_file_2023_09_02_10_00_17.wav', 'Door'), ('big_test_folder/few_garden_files/er_file_2023_09_04_7_13_08.wav', 'Owl'), ('big_test_folder/few_garden_files/er_file_2023_09_02_16_45_18.wav', 'Caw')]


In [61]:
from tqdm import tqdm
# analyze files in the folder of folders, wrap in tqdm to show progress bar

def analyze_folders(folder):
    folders = [f for f in os.listdir(folder) if not f.startswith('.') and not f.endswith('.json')]
    results = []
    for f in tqdm(folders):
        results += analyze_files_ast(folder + f + "/")
       
    return results



# method to add a column with 'MIT_AST_label' to the dataframe, allign by file name
def add_label_column(df, results):
    df['MIT_AST_label'] = None
    for file, label in results:
        df.loc[df['filename'] == file, 'MIT_AST_label'] = label
    return df

# get metadata_file and output_file from folder name
def get_metadata_output_files(folder):
    metadata_file = "metadata/" +folder[:-1] + "_metadata_birdnet_labeled.xlsx"
    output_file = metadata_file.split(".")[0] + "_mit_ast.xlsx"
    return metadata_file, output_file



folder = "garden_161220023/"
metadata_file, output_file = get_metadata_output_files(folder)
metadata_file, output_file

('metadata/garden_161220023_metadata_birdnet_labeled.xlsx',
 'metadata/garden_161220023_metadata_birdnet_labeled_mit_ast.xlsx')

In [67]:
test_folder = 'big_test_folder/'

In [68]:
m, o = get_metadata_output_files(test_folder)
m, o

('metadata/big_test_folder_metadata_birdnet_labeled.xlsx',
 'metadata/big_test_folder_metadata_birdnet_labeled_mit_ast.xlsx')

In [94]:
test_folder = 'garden_19102023/'
res = analyze_folders(test_folder)
print(res)

100%|██████████| 17/17 [14:35<00:00, 51.51s/it]

[('er_file_2023_10_19_16_55_54.wav', 'Caw'), ('er_file_2023_10_19_17_18_42.wav', 'Vehicle'), ('er_file_2023_10_19_16_41_08.wav', 'Squeak'), ('er_file_2023_10_19_16_39_04.wav', 'Vehicle'), ('er_file_2023_10_19_16_59_41.wav', 'Bird'), ('er_file_2023_10_19_16_57_50.wav', 'Bird'), ('er_file_2023_10_19_16_54_03.wav', 'Caw'), ('er_file_2023_10_19_16_57_52.wav', 'Fowl'), ('er_file_2023_10_19_17_31_29.wav', 'Vehicle'), ('er_file_2023_10_19_17_40_15.wav', 'Animal'), ('er_file_2023_10_19_17_12_35.wav', 'Bird'), ('er_file_2023_10_19_16_45_52.wav', 'Fowl'), ('er_file_2023_10_19_16_56_12.wav', 'Fowl'), ('er_file_2023_10_19_17_08_50.wav', 'Bird'), ('er_file_2023_10_19_17_15_29.wav', 'Chirp, tweet'), ('er_file_2023_10_19_17_42_05.wav', 'Speech'), ('er_file_2023_10_19_16_57_47.wav', 'Animal'), ('er_file_2023_10_19_17_26_24.wav', 'Speech'), ('er_file_2023_10_19_17_00_08.wav', 'Rustling leaves'), ('er_file_2023_10_19_16_54_12.wav', 'Caw'), ('er_file_2023_10_19_17_07_11.wav', 'Heart sounds, heartbeat'), 

In [111]:
res[0][0]    

'er_file_2023_10_19_16_55_54.wav'

In [112]:
f = pd.read_excel("metadata/garden_19102023_metadata_birdnet_labeled_mit_ast.xlsx")
#f.loc[f['filename'] == res[0][0], 'MIT_AST_label'] = res[0][1]
f['filename'][1]

'garden_19102023/160/er_file_2023_10_19_9_38_33.wav'

In [109]:
f[f['MIT_AST_label'].notnull()].head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,...,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,label,MIT_AST_label


In [99]:
# make a df out of the results
df = pd.DataFrame(res, columns=['filename', 'MIT_AST_label'])
df.head(20)

,filename,MIT_AST_label
0,er_file_2023_10_19_16_55_54.wav,Caw
1,er_file_2023_10_19_17_18_42.wav,Vehicle
2,er_file_2023_10_19_16_41_08.wav,Squeak
3,er_file_2023_10_19_16_39_04.wav,Vehicle
4,er_file_2023_10_19_16_59_41.wav,Bird
5,er_file_2023_10_19_16_57_50.wav,Bird
6,er_file_2023_10_19_16_54_03.wav,Caw
7,er_file_2023_10_19_16_57_52.wav,Fowl
8,er_file_2023_10_19_17_31_29.wav,Vehicle
9,er_file_2023_10_19_17_40_15.wav,Animal


In [100]:
# drop column 'MIT_AST_label'
df = df.drop(columns=['MIT_AST_label'])
df.head()

,filename
0,er_file_2023_10_19_16_55_54.wav
1,er_file_2023_10_19_17_18_42.wav
2,er_file_2023_10_19_16_41_08.wav
3,er_file_2023_10_19_16_39_04.wav
4,er_file_2023_10_19_16_59_41.wav


In [101]:
df = add_label_column(df, res)
df.head()

,filename,MIT_AST_label
0,er_file_2023_10_19_16_55_54.wav,Caw
1,er_file_2023_10_19_17_18_42.wav,Vehicle
2,er_file_2023_10_19_16_41_08.wav,Squeak
3,er_file_2023_10_19_16_39_04.wav,Vehicle
4,er_file_2023_10_19_16_59_41.wav,Bird


In [102]:
df.to_excel('metadata/garden_19102023_metadata_birdnet_labeled_mit_ast.xlsx')

In [59]:
def pipeline_mit_ast(folder):
    metadata_file, output_file = get_metadata_output_files(folder)
    results = analyze_folders(folder)
    df = pd.read_excel(metadata_file)
    df = add_label_column(df, results)
    df.to_excel(output_file)

In [124]:
# test the pipeline
folder = "garden_11082023/"

pipeline_mit_ast(folder)


100%|██████████| 49/49 [42:10<00:00, 51.64s/it]


In [120]:
mf = pd.read_excel("metadata/garden_16122023_metadata_birdnet_labeled_mit_ast.xlsx")
mf[mf['MIT_AST_label'].notnull()].head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sessionId,time,filename,th1,th1_value,th2,th2_value,...,windspeedAvg,tempAve,humidityAvg,winddirAvg,uvHigh,solarRadiationHigh,lon,lat,label,MIT_AST_label
0,0,0,0,test,_2023_12_16_18_02_08,garden_16122023/0/er_file_2023_12_16_18_02_08.wav,0.154655,0.22,0.921054,0.5,...,3.0,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Rustling leaves
1,1,1,1,test,_2023_12_16_18_02_11,garden_16122023/0/er_file_2023_12_16_18_02_11.wav,0.545252,0.22,0.869510,0.5,...,3.0,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Heart murmur
2,2,2,2,test,_2023_12_16_18_02_14,garden_16122023/0/er_file_2023_12_16_18_02_14.wav,0.262045,0.22,0.680537,0.5,...,3.0,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Knock
3,3,3,3,test,_2023_12_16_18_02_17,garden_16122023/0/er_file_2023_12_16_18_02_17.wav,0.465999,0.22,0.746294,0.5,...,3.0,9.1,98.0,204.0,0.0,0.0,5.462251,51.417782,NaN,Door
4,4,4,4,test,_2023_12_16_18_02_23,garden_16122023/0/er_file_2023_12_16_18_02_23.wav,0.245217,0.22,0.715698,0.5,...,3.6,9.1,98.0,193.0,0.0,0.0,5.462251,51.417782,NaN,Door
